In [ ]:
# auto_label_yolo11.py
import argparse
import os
from pathlib import Path
from typing import List
import re  # Bunu importların arasına ekle
import torch
from ultralytics import YOLO
from PIL import Image

def xyxy_to_yolo(box, img_w, img_h):
    x1, y1, x2, y2 = box
    w = max(x2 - x1, 0.0)
    h = max(y2 - y1, 0.0)
    cx = x1 + w / 2.0
    cy = y1 + h / 2.0
    # normalize
    return [
        (cx / img_w),
        (cy / img_h),
        (w / img_w),
        (h / img_h),
    ]

def list_images(folder: Path) -> List[Path]:
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
    files = [p for p in folder.rglob("*") if p.suffix.lower() in exts]
    
    return sorted(files, key=lambda x: int(re.search(r'\d+', x.stem).group()) if re.search(r'\d+', x.stem) else x.stem)

def main():

    #------------------------------ Ayarlar ------------------------------

    img_dir = Path("IMAGE_PATH")
    out_dir = Path("LABEL_OUTPUT_PATH")
    model_dir = Path("MODEL_PATH")

    args = {'conf': 0.6, 'iou': 0.2}
    #------------------------------ ------- ------------------------------

    if not img_dir.exists():
        raise SystemExit(f"Görsel klasörü bulunamadı: {img_dir}")
        

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Modeli yükle
    model = YOLO(model_dir)
    model.to(device)

    images = list_images(img_dir)
    if not images:
        raise SystemExit(f"Görsel bulunamadı: {img_dir}")

    print(f"{len(images)} görsel bulundu. Çıktılar: {out_dir.resolve()}")
    for img_path in images:
        # Görüntü boyutlarını al
        with Image.open(img_path) as im:
            img_w, img_h = im.size

        # Tahmin
        results = model.predict(
            source=str(img_path),
            conf=args['conf'],
            iou=args['iou'],
            device=device,
            verbose=False
        )

        # Her görüntü için tek sonuç nesnesi beklenir
        if len(results) == 0:
            print(f"Uyarı: Sonuç yok -> {img_path.name}")
            continue

        res = results[0]
        boxes = res.boxes  # Boxes object
        if boxes is None or len(boxes) == 0:
            # Boş txt de oluşturabilirsiniz isterseniz:
            open(out_dir / (img_path.stem + ".txt"), "w").close()
            print(f"Etiket yok (boş txt yazıldı): {img_path.name}")
            continue

        # YOLO txt satırlarını oluştur
        lines = []
        # boxes.xyxy, boxes.cls, boxes.conf mevcuttur
        xyxy = boxes.xyxy.cpu().numpy()
        cls = boxes.cls.cpu().numpy().astype(int)

        for i in range(len(xyxy)):
            c = cls[i]
            x1, y1, x2, y2 = xyxy[i]
            x, y, w, h = xyxy_to_yolo([float(x1), float(y1), float(x2), float(y2)], img_w, img_h)
            # format: class x_center y_center width height
            if c == 4:
                lines.insert(0, f"{c} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")
            else:
                lines.append(f"{c} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

        # txt'yi yaz
        label_path = out_dir / (img_path.stem + ".txt")
        with open(label_path, "w", encoding="utf-8") as f:
            f.write("\n".join(lines))

        print(f"Yazıldı: {label_path.name}  (n={len(lines)})")

    print("Bitti ✅")

if __name__ == "__main__":
    main()


38576 görsel bulundu. Çıktılar: C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\Pursu_Cam_Sise\Dataset\labels
Yazıldı: Cam_Sise0.txt  (n=25)
Yazıldı: Cam_Sise1.txt  (n=26)
Yazıldı: Cam_Sise2.txt  (n=27)
Yazıldı: Cam_Sise3.txt  (n=26)
Yazıldı: Cam_Sise4.txt  (n=29)
Yazıldı: Cam_Sise5.txt  (n=27)
Yazıldı: Cam_Sise6.txt  (n=21)
Yazıldı: Cam_Sise7.txt  (n=25)
Yazıldı: Cam_Sise8.txt  (n=19)
Yazıldı: Cam_Sise9.txt  (n=22)
Yazıldı: Cam_Sise10.txt  (n=17)
Yazıldı: Cam_Sise11.txt  (n=22)
Yazıldı: Cam_Sise12.txt  (n=16)
Yazıldı: Cam_Sise13.txt  (n=21)
Yazıldı: Cam_Sise14.txt  (n=23)
Yazıldı: Cam_Sise15.txt  (n=22)
Yazıldı: Cam_Sise16.txt  (n=21)
Yazıldı: Cam_Sise17.txt  (n=24)
Yazıldı: Cam_Sise18.txt  (n=23)
Yazıldı: Cam_Sise19.txt  (n=20)
Yazıldı: Cam_Sise20.txt  (n=15)
Yazıldı: Cam_Sise21.txt  (n=17)
Yazıldı: Cam_Sise22.txt  (n=17)
Yazıldı: Cam_Sise23.txt  (n=20)
Yazıldı: Cam_Sise24.txt  (n=23)
Yazıldı: Cam_Sise25.txt  (n=27)
Yazıldı: Cam_Sise26.txt  (n=23)
Yazıldı: Cam_Sise27.txt  (n=22)
Yazıldı:

KeyboardInterrupt: 